In [ ]:
# 数据挖掘-特征工程
#2、对数据特征进行衍生和进行特征挑选
#特征衍生
#特征挑选：分别用IV值和随机森林等进行特征选择

In [1]:
#0、准备工作-数据处理
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns 
sns.set_style('whitegrid')
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

import seaborn as sns 
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
#0-1、查看数据
data = pd.read_csv('./data/data.csv',encoding='gbk')
data.shape

(4754, 90)

In [34]:
#0-2、删除无意义的列
drop_features = ['Unnamed: 0','custid','trade_no','bank_card_no','id_name',
                   'student_feature','source','latest_query_time','loans_latest_time']
data_del_useless = data.drop(drop_features,axis=1)
data_del_useless.shape

(4423, 81)

In [35]:
#0-3\
#删除缺失值超过30%的行
data_procna = data_del_useless.dropna(axis=0,thresh=data_del_useless.shape[1]*0.7)
#删除缺失值超过30%的列
data_procna = data_procna.dropna(axis=1,thresh=data_del_useless.shape[0]*0.7)
#使用均值替换缺失值
data_procna = data_procna.fillna(data_procna.mean())
data_procna.shape

(4423, 81)

In [36]:
#0-4、数据类型转换
data_procna['reg_preference_for_trad'].value_counts()

一线城市    3172
三线城市     991
境外       133
二线城市     123
其他城市       4
Name: reg_preference_for_trad, dtype: int64

In [37]:
#若是有未知的城市类型，可换为其他城市
#data_procna['reg_preference_for_trad'].fillna('其他城市',inplace = True)
data_procna['reg_preference_for_trad'].replace({'一线城市':1,'二线城市':2,'三线城市':3,'境外':4,'其他城市':5},inplace = True)
data_procna.shape

(4423, 81)

In [38]:
data_procna['reg_preference_for_trad'].value_counts()

1    3172
3     991
4     133
2     123
5       4
Name: reg_preference_for_trad, dtype: int64

In [39]:
#0-5、查看缺失值
data_procna.isnull().sum()
#data_procna.isnull().sum().sort_values(ascending=False).head(15)

low_volume_percent                        0
middle_volume_percent                     0
take_amount_in_later_12_month_highest     0
trans_amount_increase_rate_lately         0
trans_activity_month                      0
trans_activity_day                        0
transd_mcc                                0
trans_days_interval_filter                0
trans_days_interval                       0
regional_mobility                         0
repayment_capability                      0
is_high_user                              0
number_of_trans_from_2011                 0
first_transaction_time                    0
historical_trans_amount                   0
historical_trans_day                      0
rank_trad_1_month                         0
trans_amount_3_month                      0
avg_consume_less_12_valid_month           0
abs                                       0
top_trans_count_last_1_month              0
avg_price_last_12_month                   0
avg_price_top_last_12_valid_mont

In [40]:
data_procna.shape

(4423, 81)

In [41]:
data_procna.describe()

,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
count,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,...,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000,4423.000000
mean,0.021488,0.902532,1974.756952,14.965855,0.804777,0.369008,17.618359,28.860276,21.540809,2.681438,...,3398.666064,1823.206421,9193.556410,76.138594,4.754917,5.252770,16191.634637,8015.498983,23.917929,54.865024
std,0.038201,0.140686,4005.884040,719.447970,0.196672,0.169989,4.446572,22.634800,16.240311,0.890188,...,1468.214573,579.754054,7300.776685,14.317806,2.970393,3.404822,14248.056439,5662.401622,37.515102,53.340323
min,0.000000,0.000000,0.000000,0.000000,0.120000,0.033000,2.000000,0.000000,4.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000
25%,0.010000,0.880000,0.000000,0.610000,0.670000,0.236000,15.000000,16.000000,12.000000,2.000000,...,2300.000000,1540.000000,4800.000000,77.000000,2.000000,3.000000,7800.000000,4750.000000,5.000000,10.000000
50%,0.010000,0.960000,600.000000,0.960000,0.860000,0.354000,18.000000,23.000000,17.000000,3.000000,...,3200.000000,1811.000000,7700.000000,79.000000,4.000000,5.000000,13800.000000,7050.000000,14.000000,36.000000
75%,0.020000,0.990000,2000.000000,1.580000,1.000000,0.480000,20.000000,32.000000,26.000000,3.000000,...,4300.000000,2100.000000,11700.000000,80.000000,7.000000,7.000000,20400.000000,10000.000000,24.000000,90.000000
max,1.000000,1.000000,68000.000000,47596.740000,1.000000,0.941000,42.000000,285.000000,234.000000,5.000000,...,10000.000000,6900.000000,87100.000000,87.000000,18.000000,20.000000,266400.000000,82800.000000,360.000000,323.000000


In [24]:
#2、特征衍生
#特征衍生是现有的特征进行某种组合，生成新的具有含义的特征
label = data_procna.columns
print('label=',label)
features = data_procna.drop(['status'],axis = 1, inplace=False)

label= Index(['Unnamed: 0', 'custid', 'trade_no', 'bank_card_no',
       'low_volume_percent', 'middle_volume_percent',
       'take_amount_in_later_12_month_highest',
       'trans_amount_increase_rate_lately', 'trans_activity_month',
       'trans_activity_day', 'transd_mcc', 'trans_days_interval_filter',
       'trans_days_interval', 'regional_mobility', 'repayment_capability',
       'is_high_user', 'number_of_trans_from_2011', 'first_transaction_time',
       'historical_trans_amount', 'historical_trans_day', 'rank_trad_1_month',
       'trans_amount_3_month', 'avg_consume_less_12_valid_month', 'abs',
       'top_trans_count_last_1_month', 'avg_price_last_12_month',
       'avg_price_top_last_12_valid_month', 'reg_preference_for_trad',
       'trans_top_time_last_1_month', 'trans_top_time_last_6_month',
       'consume_top_time_last_1_month', 'consume_top_time_last_6_month',
       'cross_consume_count_last_1_month',
       'trans_fail_top_count_enum_last_1_month',
       'trans_f

In [ ]:
#上述就有很多就是衍生后的特征，如latest_one_month_suc、latest_one_month_fail、latest_six_month_loan都是处理好的特征
#衍生特征一般有两种原因引起的：数据自身的变化，使数据中出现很多原来没有的特征；进行特征学习时，算法根据特征之间的某种关系，产生了衍生特征，
#有时衍生特征更能反应数据特征之间的关系 。衍生特征也要求机器学习和深度学习算法拥有更强的学习能力，即增量学习、在线学习、迁移学习。

In [ ]:
#3、特征选择( Feature Selection )也称特征子集选择( Feature Subset Selection , FSS )，或属性选择( Attribute Selection )。
#是指从已有的M个特征(Feature)中选择N个特征使得系统的特定指标最优化，是从原始特征中选择出一些最有效特征以降低数据集维度的过程,
#是提高学习算法性能的一个重要手段,也是模式识别中关键的数据预处理步骤。对于一个学习算法来说,好的学习样本是训练模型的关键。

In [46]:
y = data_procna['status']
X = data_procna.drop('status',axis=1)

#数据集切分
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)

#使用随机森林进行特征选择
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500,random_state=2018,verbose=2,oob_score=True)
rfc.fit(X_train,y_train)

#imp = rfc.feature_importances_
#imp_result = np.argsort(imp)[::-1]

#for i in range(data_procna.shape[1]):
#   print("%2d. %-*s %f"%(i+1, 30, label[i], imp[imp_result[i]]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
b

building tree 335 of 500
building tree 336 of 500
building tree 337 of 500
building tree 338 of 500
building tree 339 of 500
building tree 340 of 500
building tree 341 of 500
building tree 342 of 500
building tree 343 of 500
building tree 344 of 500
building tree 345 of 500
building tree 346 of 500
building tree 347 of 500
building tree 348 of 500
building tree 349 of 500
building tree 350 of 500
building tree 351 of 500
building tree 352 of 500
building tree 353 of 500
building tree 354 of 500
building tree 355 of 500
building tree 356 of 500
building tree 357 of 500
building tree 358 of 500
building tree 359 of 500
building tree 360 of 500
building tree 361 of 500
building tree 362 of 500
building tree 363 of 500
building tree 364 of 500
building tree 365 of 500
building tree 366 of 500
building tree 367 of 500
building tree 368 of 500
building tree 369 of 500
building tree 370 of 500
building tree 371 of 500
building tree 372 of 500
building tree 373 of 500
building tree 374 of 500


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=True, random_state=2018,
                       verbose=2, warm_start=False)

In [43]:
feature_dict = dict(zip(data_procna.columns,rfc.feature_importances_))
feature_dict

{'low_volume_percent': 0.0071457790760815765,
 'middle_volume_percent': 0.009713210926753445,
 'take_amount_in_later_12_month_highest': 0.010563532051890239,
 'trans_amount_increase_rate_lately': 0.014038360333277789,
 'trans_activity_month': 0.010429841253243262,
 'trans_activity_day': 0.014029843582706368,
 'transd_mcc': 0.010519727230514767,
 'trans_days_interval_filter': 0.012622005007532905,
 'trans_days_interval': 0.011822665507147445,
 'regional_mobility': 0.004652057486026229,
 'repayment_capability': 0.015745614116358504,
 'is_high_user': 0.00032253774268666273,
 'number_of_trans_from_2011': 0.0116346743589484,
 'first_transaction_time': 0.014230819649100902,
 'historical_trans_amount': 0.015682738638911224,
 'historical_trans_day': 0.013372821571825864,
 'rank_trad_1_month': 0.012744337984113216,
 'trans_amount_3_month': 0.01692515246441227,
 'avg_consume_less_12_valid_month': 0.0065554178567624995,
 'abs': 0.015357177372998207,
 'top_trans_count_last_1_month': 0.007764779775

In [47]:
# 包外评估
print('oob_score:',rfc.oob_score_)

oob_score: 0.7965116279069767


In [44]:
X[['take_amount_in_later_12_month_highest']].max()

take_amount_in_later_12_month_highest    68000
dtype: int64

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)
print(X_train, X_test, y_train, y_test)

      low_volume_percent  middle_volume_percent  \
4733                0.03                   0.97   
75                  0.00                   1.00   
871                 0.13                   0.71   
2917                0.01                   0.72   
1254                0.02                   0.90   
1801                0.09                   0.91   
4665                0.03                   0.97   
2848                0.00                   1.00   
565                 0.02                   0.94   
1621                0.03                   0.97   
142                 0.02                   0.98   
872                 0.01                   0.99   
1951                0.06                   0.94   
2405                0.06                   0.94   
4736                0.05                   0.95   
3942                0.00                   0.40   
2274                0.00                   1.00   
368                 0.05                   0.95   
40                  0.01       

In [56]:
#IV值
def CalcIV(Xvar,Yvar):
    N_0=np.sum(Yvar==0)
    N_1=np.sum(Yvar==1)
    N_0_group=np.zeros(np.unique(Xvar).shape)
    N_1_group=np.zeros(np.unique(Xvar).shape)
    for i in range(len(np.unique(Xvar))):
        N_0_group[i] = Yvar[(Xvar==np.unique(Xvar)[i])&(Yvar==0)].count()
        N_1_group[i] = Yvar[(Xvar==np.unique(Xvar)[i])&(Yvar==1)].count()
    iv = np.sum((N_0_group/N_0-N_1_group/N_1)*np.log((N_0_group/N_0)/(N_1_group/N_1)))
    if iv>=1.0:## 处理极端值
        iv=1
    return iv

def caliv_batch(df,Yvar):
    ivlist=[]
    for col in df.columns:
        iv=CalcIV(df[col],Yvar)
        ivlist.append(iv)
    names=list(df.columns)
    iv_df=pd.DataFrame({'Var':names,'Iv':ivlist},columns=['Var','Iv'])
    return iv_df,ivlist

im_iv, ivl = caliv_batch(X,y)

In [57]:
print(im_iv, ivl)

                                       Var        Iv
0                       low_volume_percent  1.000000
1                    middle_volume_percent  1.000000
2    take_amount_in_later_12_month_highest  1.000000
3        trans_amount_increase_rate_lately  1.000000
4                     trans_activity_month  1.000000
5                       trans_activity_day  1.000000
6                               transd_mcc  1.000000
7               trans_days_interval_filter  1.000000
8                      trans_days_interval  1.000000
9                        regional_mobility  0.008366
10                    repayment_capability  1.000000
11                            is_high_user  0.004533
12               number_of_trans_from_2011  1.000000
13                  first_transaction_time  1.000000
14                 historical_trans_amount  1.000000
15                    historical_trans_day  1.000000
16                       rank_trad_1_month  0.128384
17                    trans_amount_3_month  1.

In [58]:
#过滤IV值小于0.02的特征
threshold = 0.02
threshold2 = 0.8
data_index = []
for i in range(len(ivl)):
    if im_iv['Iv'][i]< threshold or im_iv['Iv'][i] > threshold2:
        data_index.append(im_iv['Var'][i])
print(data_index)

['low_volume_percent', 'middle_volume_percent', 'take_amount_in_later_12_month_highest', 'trans_amount_increase_rate_lately', 'trans_activity_month', 'trans_activity_day', 'transd_mcc', 'trans_days_interval_filter', 'trans_days_interval', 'regional_mobility', 'repayment_capability', 'is_high_user', 'number_of_trans_from_2011', 'first_transaction_time', 'historical_trans_amount', 'historical_trans_day', 'trans_amount_3_month', 'avg_consume_less_12_valid_month', 'abs', 'avg_price_last_12_month', 'avg_price_top_last_12_valid_month', 'reg_preference_for_trad', 'trans_top_time_last_1_month', 'trans_top_time_last_6_month', 'consume_top_time_last_1_month', 'consume_top_time_last_6_month', 'cross_consume_count_last_1_month', 'trans_fail_top_count_enum_last_1_month', 'trans_fail_top_count_enum_last_6_month', 'trans_fail_top_count_enum_last_12_month', 'consume_mini_time_last_1_month', 'max_cumulative_consume_later_1_month', 'max_consume_count_later_6_month', 'railway_consume_count_last_12_month'

In [59]:
data_procna.drop(data_index, axis=1, inplace=True)
data_procna.shape

(4423, 4)

In [63]:
data_procna

,rank_trad_1_month,top_trans_count_last_1_month,status,consfin_product_count
0,0.40,0.15,1,2.0
1,0.35,0.05,0,6.0
2,1.00,0.65,1,1.0
3,0.15,0.05,0,5.0
4,0.65,1.00,1,2.0
5,0.20,0.05,0,12.0
6,1.00,1.00,0,2.0
8,0.35,0.05,0,11.0
9,0.75,1.00,1,0.0
10,0.75,0.65,1,8.0
